_____________
# 04. Iterators | Generators | `lambda` | `map()` | `filter()`
In the following notebook, we will introduce something that you've actually came across in this course a few times already, ***iterators*** and ***generators***, as well as cover a number of additional function topics, such as `map()`, `filter()`, and `lambda`.
_____________
# Iterators
An important part of the scientific approach is repeating calculations over and over again, preferably in an automated fashion. Python allows that by the use of *iterators*, which are objects containing a countable number of values that you can traverse through.

More technically, in Python, an iterator is an object which implements the iterator protocol consisting of the `__iter__()` and `__next()__ methods`.

### Iterator vs. Iterable
Lists, tuples, dictionaries, and sets are all iterable objects. They are iterable containers which you can get an iterator from.

All these objects have an `iter()` method which is used to get an iterator:

In [ ]:
mytuple = ("apple", "banana", "cherry")
myit = iter(mytuple)

In [ ]:
print(next(myit))

In [ ]:
print(next(myit))

In [ ]:
print(next(myit))

Strings are also iterables, containing sequences of chars:

In [ ]:
mystr = "banana"
myit = iter(mystr)

In [ ]:
print(next(myit))

In [ ]:
print(next(myit))

Another example of an iterable that is commonly mistaken as a list, `range()`:

In [ ]:
range(10)

In [ ]:
r = range(10)
myit = iter(r)

In [ ]:
print(next(myit))

In [ ]:
print(next(myit))

### Looping Through an Iterator
As observed previously, we may loop through any iterable object using the `for` cycle:

In [ ]:
for i in range(10):
    print(i, end=' ')

Iterators are perhaps most easily understood in the concrete case of iterating through a list.
Consider the following:

In [ ]:
L = [2, 4, 6, 8, 10]

In [ ]:
for value in L:
    # do some operation
    print(value + 1, end=' ')

Whilst the ``for x in y`` syntax allows us to repeat some operation for each value in the list it is not what's *really* happening.

When you write something like "``for val in L``", the Python interpreter checks whether the object `L` has an *iterator* interface, which you can check yourself with the built-in ``iter`` function:

In [ ]:
iter(L)

The ``iter`` object is a container that gives you access to the next object for as long as it's valid, which can be seen with the built-in function ``next``:

In [ ]:
I = iter([2, 4, 6, 8, 10])

In [ ]:
print(next(I))

In [ ]:
print(next(I))

In [ ]:
print(next(I))

Essentially, that provides the functionality required by the ``for`` loop.

Like a list, `range` hides an iterator, so Python knows to treat it as if it was a list.

In [ ]:
range(10)

In [ ]:
iter(range(10))

In [ ]:
for i in range(10):
    print(i, end=' ')

The benefit of the ***iterator indirection*** is that ***the full list is never explicitly created!*** 

We can see this by doing a range calculation that would overwhelm our system memory if we actually instantiated it

In [ ]:
N = 10 ** 12
for i in range(N):
    if i >= 10: break
    print(i, end=', ')

If ``range`` actually created that list of one trillion values, it would occupy tens of terabytes of machine memory: a waste, given the fact that we're ignoring all but the first 10 values!

In fact, there's no reason that iterators ever have to end at all!
Python's ``itertools`` library contains a ``count`` function that acts as an infinite range:

In [ ]:
from itertools import count

for i in count():
    if i >= 10:
        break
    print(i, end=', ')

## Specialized Iterators: `itertools`
The ``itertools`` module contains a variety of useful iterators, so you might want to explore [its documentation](https://docs.python.org/3.5/library/itertools.html) yourself when you have some extra time. For example:

``itertools.permutations`` function iterates over all permutations of a sequence:

In [ ]:
from itertools import permutations
p = permutations(range(3))
print(*p)

``itertools.combinations`` function iterates over all unique combinations of ``N`` values within a list:

In [ ]:
from itertools import combinations
c = combinations(range(4), 2)
print(*c)

The ``product`` iterator iterates over all sets of pairs between two or more iterables:

In [ ]:
from itertools import product
p = product('ab', range(3))
print(*p)

# Generators | Generator Expressions | Generator Comprehensions
Python ***generators*** are a kind of iterables that you can iterate over only once. They are easy to implement, but often difficult to understand, so we will start by simply comparing them with something you know much better,  list comprehensions, starting from the syntax: 

    List comprehensions use square brackets, generator expressions use parentheses.

In [ ]:
# This is a list comprehension
cubes = [n ** 3 for n in range(12)]
cubes

This is a generator expression representing the comprehension above, also known as *generator comprehension*:

In [ ]:
cubes = (n ** 3 for n in range(12))
cubes

In order to print the generator expression contents, you can pass it to the `list` constructor:

In [ ]:
list(cubes)

    A list is a collection of values, while a generator is a recipe for producing values
    
That is, when creating a list, you are actually building a collection of values, and there is some memory cost associated with that, but when creating a generator, you are only creating a recipe for producing these values (whenever necessary). Due to this, the generators are more memory and|or computationally efficient, also being unlimited in their size. 

Both use the same iterator interface:

In [ ]:
cubes_l = [n ** 3 for n in range(12)]
for val in cubes_l:
    print(val, end=' ')

In [ ]:
cubes_g = (n ** 3 for n in range(12))
for val in cubes_g:
    print(val, end=' ')

In [ ]:
%%timeit
cubes = [x ** 3 for x in range(100000)]

%%timeit
cubes = (x ** 3 for x in range(100000))

An example of an infinite generator expression can be created using the ``count`` iterator defined in ``itertools``:

In [ ]:
from itertools import count
count()

In [ ]:
for i in count():
    print(i, end=' ')
    if i >= 10: break

The ``count`` iterator will go on counting forever until you tell it to stop; this makes it convenient to create generators that will also go on forever:

In [ ]:
factors = [2, 3, 5, 7]
G = (i for i in count() if all(i % n > 0 for n in factors))
for val in G:
    print(val, end=' ')
    if val > 40: 
        break

    
    A list can be iterated multiple times; a generator expression is single-use

E.g., you can go through the list multiple times:

In [ ]:
L = [n ** 2 for n in range(12)]
for val in L:
    print(val, end=' ')

In [ ]:
for val in L:
    print(val, end=' ')

But you can't do the same with a generator (it is consumed):

In [ ]:
G = (n ** 2 for n in range(12))
list(G)

In [ ]:
list(G)

This might be useful when working with large collections of data on disk, as you can easily analyze them in batches, letting the generator itself keep the track of the files that are yet to be seen. What is more, this also means that an iteration can be stopped and started again:

In [ ]:
G = (n**2 for n in range(12))
for n in G:
    print(n, end=' ')
    if n > 30: break

print("\ndoing something in between")

for n in G:
    print(n, end=' ')

## Generator Functions:  ``yield``
As observed previously, while list comprehensions work well in simple scenarions, the standard `for` can be better in more complicated situations. Similarly, we can make the generators more sophisticated using the ``yield`` statement:

In [ ]:
def simple_range(n):
    for i in range(n):
        yield i

When an iteration over a set of item starts using the for statement, the generator is run. Once the generator's function code reaches a `yield` statement, the generator `yields its execution back to the for loop, returning a new value from the set`.

In [ ]:
x = simple_range(4)

In [ ]:
x

In [ ]:
list(x), list(x)

And again, 2 ways of constructing the same list:

In [ ]:
L1 = [n ** 2 for n in range(12)]

L2 = []
for n in range(12):
    L2.append(n ** 2)

print(L1)
print(L2)

2 ways of constructing equivalent generators:

In [ ]:
G1 = (n ** 2 for n in range(12))

def gen():
    for n in range(12):
        yield n ** 2

G2 = gen()
print(*G1)
print(*G2)

A generator function is a function that use ``yield`` to yield a (potentially infinite) sequence of values instead of `return` that returns a value once. Just as in generator expressions, the state of the generator is preserved between partial iterations, but if we want a fresh copy of the generator we can simply call the function again.

Again, Python modularity allows to combine `yield` with `while`, `for`, or other control flow statements, such as `break`. pushing the boundaries for Python expressiveness even further.

In [ ]:
from random import random

def yield_until_larger(target=0.99):
    while True:
        x = random()
        yield x
        if x > target:
            break

In [ ]:
list(yield_until_larger())

In [ ]:
list(yield_until_larger())

In [ ]:
sum(yield_until_larger())

Here's another example of how `yield` can be used to re-create the range function:

In [ ]:
def xrange(start=0, end=None, step=1):
    if end is None:
        end, start = start, 0
    while start < end:
        yield start
        start += step

In [ ]:
for x in xrange(0, 10, 2):
    print(x)
    if x >= 10:
        break

### `Step-by-Step` Prime Number Generator
This is a step-by-step guide for creating a function to generate an unbounded series of prime numbers using the [*Sieve of Eratosthenes*](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes) algorithm:

In [ ]:
# Generate a list of candidates
L = [n for n in range(2, 40)]
print(L)

In [ ]:
# Remove all multiples of the first value
L = [n for n in L if n == L[0] or n % L[0] > 0]
print(L)

In [ ]:
# Remove all multiples of the second value
L = [n for n in L if n == L[1] or n % L[1] > 0]
print(L)

In [ ]:
# Remove all multiples of the third value
L = [n for n in L if n == L[2] or n % L[2] > 0]
print(L)

If we repeat this procedure enough times on a large enough list, we can generate as many primes as we wish.

Let's encapsulate this logic in a generator function:

In [ ]:
def gen_primes(N):
    """Generate primes up to N"""
    primes = set()
    for n in range(2, N):
        if all(n % p > 0 for p in primes):
            primes.add(n)
            yield n

print(*gen_primes(100))

### `Exercise 1 - Fibonacci Generator`
Create an infinite generator for the fibonacci series:

    fibo[0]   = 0
    fibo[1]   = 1
    fibo[n+2] = fibo[n+1]


In [ ]:
# YOUR CODE GOES HERE

_____________
# `lambda` Expressions
A lambda function is a small anonymous function that can take any number of arguments: 

***`lambda`*** *arguments*: *expression*

Here is a lambda function that adds 10 to the number passed in as an argument:

In [ ]:
x = lambda a : a + 10
x

In [ ]:
x(5)

And another one that multiplies argument a with argument b:

In [ ]:
x = lambda a, b : a * b
x

In [ ]:
x(5, 6) 

In [ ]:
def times2(x):
    return x * 2

In [ ]:
times2(2)

In [ ]:
times2([111, 222])

In [ ]:
times2('We don\'t need no education')

In [ ]:
x = lambda var: var*2
x

In [ ]:
x(2)

In [ ]:
x([111,222])

### `Exercise 2 - Lambda Sorting Tuples`
Sort a list of tuples using a `lambda` expression.

E.g.,

Input:

    marks = [('EN', 80), ('IT', 88), ('DE', 95)]
    
Output:
    
    Sorting the tuple list:

    [('EN', 80), ('IT', 88), ('DE', 95)]    

In [ ]:
# YOUR CODE GOES HERE

### `Exercise 3 - Lambda Sorting Dict`
Sort a list of dictionaries using a `lambda` expression.

Input:

    models = [{'make':'Nokia', 'model':216, 'color':'Black'}, 
              {'make':'Mi Max', 'model':'2', 'color':'Gold'}, 
              {'make':'Samsung', 'model': 7, 'color':'Blue'}]
                 
Output:
    

    Sorting the dictionary list :
    [{'make': 'Nokia', 'model': 216, 'color': 'Black'}, {'make': 'Samsung', 'model': 7, 'color': 'Blue'}, {'make': 'Mi Max', 'model': 2, 'color': 'Gold'}]
 

In [ ]:
# YOUR CODE GOES HERE

### `Exercise 4 - Date Time Lambda`
Extract year, month, date and time using Lambda:

Input:

    datetime.datetime(2020, 3, 28, 1, 3, 54, 121736)
    
Output:
    
    Datetime: 2020-03-28 01:06:28.325593
    Year    : 2020
    Month   : 3
    Day     : 28
    Now     : 01:06:28.325593

In [ ]:
import datetime
now = datetime.datetime.now()
now

In [ ]:
??datetime

In [ ]:
# YOUR CODE GOES HERE

### `Exercise 5 - Lambda Printing`
Create a lambda named `print2`, which works the same way as `print`, but always uses tab as a separator.

`Hint` You will need the `*` operators.

In [ ]:
# YOUR CODE GOES HERE

___________
# Map & Filter

`Map` & `Filter` allow the programmer to write simpler, shorter code, without thinking about loops or branching. This way of coding is also known as functional programming, since we apply a function across a number of iterables in one full swoop. Both of these functions come built-in with Python (in the __builtins__ module) and require no importing.

## Map
The `map()` function in python has the following syntax:

    map(func, *iterables)

Where `func` is the function to be applied on each of the iterable elements (as many as there are). Note the use of `*`, for an arbitrary number of arguments. Additional notes:

- In Python 3, `map()` returns a map object which is a generator object.
- To get the result as a list, `list()` function can be called on the map object, e.g. `list(map(func, *iterables))`.
- The number of arguments to func must be the number of iterables listed.

Let's say you have a list (iterable) of your chosen laptop brands and want to uppercase them. In traditional Python, you would do something like this:

In [ ]:
laptops = ['acer', 'asus', 'macbook', 'lg']
[l.upper() for l in laptops]

Alternatively, you can do the same using the `map()` function:

In [ ]:
laptops = ['acer', 'asus', 'macbook', 'lg']
list(map(str.upper, laptops))

`func` corresponds to virtually any function, so we can also `map()` our own `def`initions:

In [ ]:
seq = range(11, 33, 3)
seq = list(seq)
seq

In [ ]:
def times2(x):
    return x * 2

map(times2, seq)

In [ ]:
list(map(times2, seq))

In [ ]:
def squared(x):
    return x ** 2

list(map(squared, seq))

We can also combine `map()` with `lambda`:

In [ ]:
list(map(lambda x: x * 2, seq))

### `Exercise 6 - Map Stats`
Assume that you gathered a list of stats (floats) about your working routines (be it the averages of customers that joined the system during each day of the month, the sums of money transactions, anything of the kind):

    numbers = [0.356, 11.12, 16.534238, 3.2456, 9.1, 5, 13.6]
    
You want to round these numbers to have the same decimal placing via the `map()` function and the `round()` Python built-in. 

Now change the functions, so that each of the elements is rounded to the position of it in the list. That is, round up the first element in the list to one decimal place, the second element in the list to two decimal places, the third element in the list to three decimal places, etc. 

`Note` Round function requires two arguments, so we need to pass in two iterables.

In [ ]:
numbers = [0.356, 11.12, 16.534238, 3.2456, 9.1, 5, 13.6]
result = list(map(round, numbers, range(1,7)))
print(result)

### `Exercise 7 - Map Lambda Zip...?`
Create a custom `zip()` function using `map()` and `lambda()`:

    s = ['a', 'b', 'c', 'd', 'e']
    n = [1,2,3,4,5]
    
    print(zip(s,n))
    
    [('a', 1), ('b', 2), ('c', 3), ('d', 4), ('e', 5)]

In [ ]:
# YOUR CODE GOES HERE

__________________
# `filter()`
Having the syntax of the form: 

    filter(func, iterable)

`filter()`then passes each element in the iterable through some function that requires to return boolean values, "filtering" away those that are false. In contrast, `map()` passes each element in the iterable through a function and returns the result of all elements having passed through the function.

There are a few points worth remembering when thinking about `filter()`:

- Unlike `map()`, only one iterable is required.
- The `func` argument is required to return a boolean type. If it doesn't, filter simply returns the iterable passed to it. Also, as only one iterable is required, it's implicit that func must only take one argument.
- `filter` passes each element in the iterable through `func` and returns only the ones that evaluate to `True`.

Here is a simple example of a function filtering a list of numbers:

In [ ]:
scores = [66, 90, 68, 59, 76, 60, 88, 74, 81, 65]

def is_A_student(score):
    return score > 75

over_75 = list(filter(is_A_student, scores))
over_75

And here is a palindrome detector using `filter` combined with `lambda`:

In [ ]:
dromes = ("demigod", "rewire", "madam", "anutforajaroftuna", "kiosk")
palindromes = list(filter(lambda word: word == word[::-1], dromes))
palindromes

And a few examples more just to make it even clearer:

In [ ]:
seq

In [ ]:
list(filter(lambda x: x > 20, seq))

In [ ]:
list(filter(lambda x: 20 < x < 30, seq))

In [ ]:
input_value = range(10, 100, 2)

In [ ]:
list(filter(lambda x: x % 3 == 0, input_value))

In [ ]:
[x for x in input_value if x % 3 == 0]

In [ ]:
from time import time

In [ ]:
input_value = range(10, 100000, 2)

In [ ]:
%%time
list(filter(lambda x: x % 3 == 0, input_value))

In [ ]:
%%time
[x for x in input_value if x % 3 == 0]

### `Exercise 8 - Is Even`
Create a function `is_even` that takes a list of numbers and returns a new list with even numbers replaced by `True` and odd numbers replaced by `False`:

Input:
    
    is_even([1, 2, 3, 4, 4])

Output:

    [False, True, False, True, True]

In [ ]:
# YOUR CODE GOES HERE

### `Exercise 9 - Custom Encrypt!`
Create a function `encrypt(s, X, Y, Z)` that a string as an input:

    `The low tide rises at 6pm on the 10th and at 7pm on the 21st, message 24.058,55.25`
    
- odd numeric characters are converted to their corresponding letters of the alphabet + X 
- even numeric characters are converted to their corresponding letters of the alphabet - X 
- alpha characters are converted to their numerical representation, where and odds get + Y, evens get -Y.
- punctuation symbols are converted to their numerical representations, the odds get + Z, evens -Z.

In [ ]:
# YOUR CODE GOES HERE

### `Exercise 10 - Custom Decrypt!`
Create a decryption function that returns the original string representation when given the encrypted version as the input and the XYZ code.

In [ ]:
# YOUR CODE GOES HERE

# Modules & Packages
Modular programming refers to the process of breaking a large, unwieldy programming task into smaller and more manageable subtasks. In Python, we usually separate batches of related functions into a number of individual modules, which can then be combined together to create a larger application, such as a package. 

There are several advantages to modularizing code in a large application:

    ***Simplicity***: Rather than focusing on the entire problem at hand, a module typically focuses on one relatively 1small portion of the problem. If you’re working on a single module, you’ll have a smaller problem domain to wrap your head around. This makes development easier and less error-prone.

    ***Maintainability***: Modules are typically designed so that they enforce logical boundaries between different problem domains. If modules are written in a way that minimizes interdependency, there is decreased likelihood that modifications to a single module will have an impact on other parts of the program. (You may even be able to make changes to a module without having any knowledge of the application outside that module.) This makes it more viable for a team of many programmers to work collaboratively on a large application.

    ***Reusability***: Functionality defined in a single module can be easily reused (through an appropriately defined interface) by other parts of the application. This eliminates the need to recreate duplicate code.

    ***Scoping***: Modules typically define a separate namespace, which helps avoid collisions between identifiers in different areas of a program. (One of the tenets in the Zen of Python is Namespaces are one honking great idea—let’s do more of those!)

There are actually three different ways to define a module in Python, but all if them are ***loaded using the `import`*** statement:

1. A module can be written in Python itself.
2. A module can be written in C and loaded dynamically at run-time, like the re (regular expression) module.
3. A built-in module is intrinsically contained in the interpreter, like the itertools module.

Here, we will mostly focus on 1 and 3, the modules that are written in Python or the built-ins, such as `math` or `itertools`.

### Explicit module import
Explicit import of a module preserves the module's content in a namespace.
The namespace is then used to refer to its contents with a "``.``" between them.
For example, here we'll import the built-in ``math`` module and compute the cosine of pi:

In [ ]:
import math
math.cos(math.pi)

In cases where the model name is very long, it might be also useful to use an alias for it:
    
    import ... as ...
    
In fact, some of the more popular libraries have well known conventions for the aliases to shorten the module names. E.g., the NumPy (Numerical Python) package, a popular third-party package useful for data science, is by convention imported under the alias ``np``:

In [ ]:
import numpy as np
np.cos(np.pi)

### Explicit import of module contents

Sometimes rather than importing the module namespace, you would just like to import a few particular items from the module. This can be done with the "``from ... import ...``" pattern.

For example, we can import just the ``cos`` function and the ``pi`` constant from the ``math`` module:

In [ ]:
from math import cos, pi
cos(pi)

### Implicit import of module contents

Finally, it is sometimes useful to import the entirety of the module contents into the local namespace.
This can be done with the "``from ... import *``" pattern:

In [ ]:
from math import *
sin(pi) ** 2 + cos(pi) ** 2

However, this should be used rarely, if at all, since these may overwrite function names that you do not intend to overwrite, and the implicitness of the statement makes it difficult to determine what has changed.

For example, Python has a built-in ``sum`` function that can be used for various operations:

In [ ]:
help(sum)

We can use this to compute the sum of a sequence, starting with a certain value (here, we'll start with ``-1``):

In [ ]:
sum(range(5), -1)

Now observe what happens if we make the *exact same function call* after importing ``*`` from ``numpy``:

In [ ]:
from numpy import *

In [ ]:
sum(range(5), -1)

The result is off by one!
The reason for this is that the ``import *`` statement *replaces* the built-in ``sum`` function with the ``numpy.sum`` function, which has a different call signature: in the former, we're summing ``range(5)`` starting at ``-1``; in the latter, we're summing ``range(5)`` along the last axis (indicated by ``-1``).

This is the type of situation that may arise if care is not taken when using "``import *``" – for this reason, it is best to avoid this unless you know exactly what you are doing.

## Importing from Python's Standard Library

Python's standard library contains many useful built-in modules, which you can read about fully in [Python's documentation](https://docs.python.org/3/library/).
Any of these can be imported with the ``import`` statement, and then explored using the help function seen in the previous section.
Here is an extremely incomplete list of some of the modules you might wish to explore and learn about:

- ``os`` and ``sys``: Tools for interfacing with the operating system, including navigating file directory structures and executing shell commands
- ``math`` and ``cmath``: Mathematical functions and operations on real and complex numbers
- ``itertools``: Tools for constructing and interacting with iterators and generators
- ``functools``: Tools that assist with functional programming
- ``random``: Tools for generating pseudorandom numbers
- ``pickle``: Tools for object persistence: saving objects to and loading objects from disk
- ``json`` and ``csv``: Tools for reading JSON-formatted and CSV-formatted files.
- ``urllib``: Tools for doing HTTP and other web requests.

You can find information on these, and many more, in the Python standard library documentation: https://docs.python.org/3/library/.

## Importing from Third-Party Modules
These can be imported just as the built-in modules, but they have to be installed on your system first. The standard registry for such modules is the [Python Package Index](http://pypi.python.org/) (*PyPI* for short), and Python comes with a program ``pip`` (a recursive acronym meaning "pip installs packages"), which will automatically fetch packages released and listed on PyPI.

For example, if you'd like to install the ``autotime``, all that is required is to type the following at the command line:
```
$ pip install autotime
```
The source code for the package will be automatically downloaded from the PyPI repository, and the package installed in the standard Python path (assuming you have permission to do so on the computer you're using).